Let's start by importing RTS-96 data:

In [1]:
####### IMPORTS: DATA ACQUISITION ########
include("../src/tmp_inst_rts96.jl")

psData = psDataLoad()

# unpack psDL (boilerplate):
(Sb,f,t,r,x,b,Y,bustype,
Gp,Gq,Dp,Dq,Rp,Rq,
Pmax,Pmin,Qmax,Qmin,Plim,
Vg,Vceiling,Vfloor,
busIdx,N,Nr,Ng,k) = unpack_psDL(psData)

####### IMPORTS: INSTANTON CODE ########
# Get instanton code into workspace:
include("../src/TemporalInstanton.jl")
include("../src/TrustRegionSubproblem.jl")

using TemporalInstanton, TrustRegionSubproblem

####### DATA LINKING ########
# Static
Ridx = find(Rp) # Vector of renewable nodes
Y = full(Y) # Full admittance matrix (ref not removed)
ref = 1 # Index of ref node
k = k # Conventional generator participation factors
tau = 1. # Heating constant
G0 = [Gp,Gp.*0.95,Gp.*0.9]
D0 = [Dp,Dp.*0.95,Dp.*0.9]
P0 = [Rp,Rp,Rp]
kQobj = 0 # Should always be zero; objective has no constant term
c = 10. # RHS of line temperature constraint
kQtheta = -c # Move to LHS so we have Q(z) = 0

T = int(length(G0)/size(Y,1))

# Changing:
lines = [(f[i],t[i]) for i in 1:length(f)];

# For a single line:
line = lines[48]

# Create instance of instanton problem
Qobj,A,b,Qtheta = temporalInstanton(Ridx,Y,ref,k,tau,line,G0,P0,D0)

(
279x279 Array{Float64,2}:
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  

In [2]:
# Build obj and constraint quadratics:
G_of_x = (Qobj,0,kQobj)
Q_of_x = (Qtheta,0,kQtheta)

(
279x279 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  

Ian's most recent set of notes partitions the vector of variables $x$ into three parts:

* $x_1\in\mathbb{R}^{N_rT}$ contains wind deviations

* $x_2\in\mathbb{R}^{(N+1)T}$ contains angle and mismatch variables

* $x_3\in\mathbb{R}^T$ contains angle difference variables involved in line temperature calculation

With this notation, the problem becomes

\begin{align}
&& \min~ & x_1^\top Q_x x_1 \\
s.t. && Ax &= b \\
&& x_3^\top x_3 &= c
\end{align}

where $x=[x_1~x_2~x_3]^\top$.

## Translation

As always, the first step in solving our problem is translation. We need to go from $Ax=b$ to $Ay=0$, where $y=x-x^*$ and $x^*$ is the translation. It is easy to find and translate by a point in the set $\{x:Ax=b\}$, but we have an additional requirement: We don't want to introduce any linear term to the norm constraint.

\begin{align}
&& \min~ & x_3^\top x_3 \\
s.t. && A_1x_1 + A_2x_2 + A_3x_3 &= b
\end{align}

How do we partition $A$?

* $A_1$ contains columns of $A$ corresponding to wind deviations. The indices of these columns are indices for which $Q_{obj}$ is nonero.

* $A_2$ contains columns corresponding to angles and mismatches. These are all the columns remaining after $A_1$ and $A_3$ are removed.

* $A_3$ contains columns corresponding to angle difference variables. This is just the last $T$ columns of $A$.

This function partitions $A$:

In [3]:
function partition_A(A,Qobj,T)
    """ Return A1, A2, A3 where
    A1 corresponds to wind
    A2 corresponds to angles + mismatch
    A3 corresponds to angle difference vars
    """
    m,n = size(A)
    idx1 = find(diag(Qobj))
    idx2 = setdiff(1:n-T,idx1)
    idx3 = n-T+1:n
    idx = [idx1,idx2,idx3]
    
    A1 = A[:,idx1]
    A2 = A[:,idx2]
    A3 = A[:,idx3]
    return A1,A2,A3,idx1,idx2,idx3
end

partition_A (generic function with 1 method)

Now that $A$ is partitioned, we can find $x_1^*$, $x_2^*$, and $x_3^*$. To avoid messing up the norm, we need $x_3^*=0$. With that constraint satisfied, we just need $A_1x_2 + A_2x_2 = b$, so let's use `[A1 A2]\b` to obtain min-norm $x_1^*$ and $x_2^*$.

In [4]:
function find_x_star(Qobj,A,b,T)
    A1,A2,A3,idx1,idx2,idx3 = partition_A(A,Qobj,T)
    x_star = zeros(size(A,2))
    x_star[[idx1,idx2]] = [A1 A2]\b
    return x_star
end

find_x_star (generic function with 1 method)

The translation itself is a change of variables from $x$ to $y = x - x^*$. The translated problem is

\begin{align}
&& \min~ & y_1^\top Q_x y_1 + 2 y_1^\top x_1^* \\
s.t. && Ay &= 0 \\
&& y_3^\top y_3 &= c
\end{align}

In [5]:
# Partition A;
A1,A2,A3,idx1,idx2,idx3 = partition_A(A,Qobj,T)

# Find translation point:
x_star = find_x_star(Qobj,A,b,T)

# Translate quadratics:
G_of_y = translate_quadratic(G_of_x,x_star)
Q_of_y = translate_quadratic(Q_of_x,x_star)

(
279x279 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  

## Kernel "rotation"

Now that the problem has been translated, we replace $y$ by $Nz$, where $N=[v_1~v_2~\ldots~v_k]\in\mathbb{R}^{n\times k}$, which will reduce the problem dimension from $n$ variables to $k$ variables.

From Ian's notes:

\begin{align}
\begin{bmatrix} y_1 \\ y_2 \\ y_3 \end{bmatrix}
&= \begin{bmatrix} N_1 \\ N_2 \\ N_3 \end{bmatrix} z
\end{align}

* The objective becomes $\min z^\top \hat{Q}_{obj} z + 2z^\top N_1x_1^*$, where $\hat{Q}_{obj} = N_1^\top Q N_1$

* The temperature constraint becomes $z^\top N_3^\top N_3 z = c$.

The following method rotates a quadratic:

In [56]:
function rotate_quadratic(G_of_x,R)
    """ Rotate quadratic G_of_x by
    rotation matrix R.
    """
    G,g,kg = G_of_x
    return (R*G*R',R*g,kg)
end

rotate_quadratic (generic function with 1 method)

Here we find and partition $N$, then rotate our objective and constraint quadratics by it:

In [7]:
N = kernel_rotation(A)[:,1:size(A,2) - rank(A)] # take only first k cols

N1,N2,N3 = N[idx1,:],N[idx2,:],N[idx3,:] # partition N

G_of_z = rotate_quadratic(G_of_y,N')
Q_of_z = rotate_quadratic(Q_of_y,N')

(
54x54 Array{Float64,2}:
  8.57715e-6  -7.83568e-6   8.32729e-6  …  -4.07499e-6  -9.85393e-6
 -7.83568e-6   1.91934e-5   5.16678e-6      3.09765e-6   1.42626e-5
  8.32729e-6   5.16678e-6   3.24455e-5     -2.57993e-6  -2.01826e-6
 -1.53017e-6   1.33535e-5   8.37457e-6     -4.28291e-7   6.46892e-6
  3.71994e-7   1.28016e-5   2.91055e-7     -3.50642e-6   2.76647e-6
 -2.11083e-7   1.47391e-5  -1.14544e-5  …  -5.69497e-6   1.74543e-6
 -2.1289e-6    1.01688e-5   2.87208e-5      4.74971e-6   1.00533e-5
  3.44011e-6   8.8652e-6    3.51056e-6     -4.63256e-6  -9.91135e-7
 -3.34661e-6   1.28924e-5   7.6256e-6       1.16143e-6   8.22289e-6
 -4.765e-6     1.32255e-5   1.22824e-5      3.21765e-6   1.07152e-5
 -1.75752e-6   9.48476e-6   2.5917e-5   …   4.06274e-6   8.96682e-6
 -2.55424e-6   1.02789e-5   1.16742e-5      1.88109e-6   7.44803e-6
 -7.16027e-6   1.42599e-5   9.35653e-6      4.53344e-6   1.30762e-5
  ⋮                                     ⋱                          
 -1.52987e-6  -1.7632e

## Constraint Eigendecomposition

Perform an Eigendecomposition: $$N_3^\top N_3 = UDU^\top$$

In [52]:
D,U = eig(Q_of_z[1])
D = round(D,10)

54-element Array{Float64,1}:
 -0.0        
 -0.0        
 -0.0        
 -0.0        
 -0.0        
 -0.0        
 -0.0        
 -0.0        
 -0.0        
 -0.0        
 -0.0        
 -0.0        
 -0.0        
  ⋮          
  0.0        
  0.0        
  0.0        
  0.0        
  0.0        
  0.0        
  0.0        
  0.0        
  0.0        
  0.000320023
  0.000320023
  0.000320023

Now let $\hat{z} = U^\top z$ so that

\begin{align}
z^\top N_3^\top N_3 z &= \hat{z}^\top D\hat{z}
\end{align}

$D$ has at most $T$ nonzero elements, because rank$(N
_3) \leq T$ by virtue of its dimension. It will look like this:

\begin{align}
D &= \begin{bmatrix} 0 & 0 \\ 0 & \hat{D} \end{bmatrix}
\end{align}

Now let $w = [w_1~w_2]^\top$ and $\hat{z}=[\hat{z}_1~\hat{z}_2]^\top$, and relate them as follows:

\begin{align}
\begin{bmatrix} w_1 \\ w_2 \end{bmatrix} &=
\begin{bmatrix} I & 0 \\ 0 & \hat{D}^{1/2} \end{bmatrix}
\begin{bmatrix} \hat{z}_1 \\ \hat{z}_2 \end{bmatrix} \\
\implies w &= K\hat{z} = KU^\top z
\end{align}

In [53]:
function return_K(D)
    K = ones(length(D))
    K[find(D)] = sqrt(D[find(D)])
    K = diagm(K)
    return K
end

return_K (generic function with 1 method)

Then

\begin{align}
\hat{z}^\top D\hat{z} &= \hat{z}_1\hat{D}^{1/2}\hat{D}^{1/2}\hat{z} \\
&= w_1^\top w_1
\end{align}

Note that $z = UK^{-1}w$ because $UU^\top = I$. Changing from $z$ to $w$ is equivalent to rotating by $(UK^{-1})^\top$.

Substituting into the cost function:

$$ z^\top \hat{Q}_{obj}z = w^\top K^{-1}U^\top \hat{Q}_{obj}UK^{-1}w + 2z^\top K^{-1}U^\top N_1x_1^* = w^\top Bw + w^\top b$$

With $w$, $B$, and $b$ defined, the optimization problem becomes

\begin{align}
&& \min~ w^\top Bw + w^\top b \\
s.t. && w_2^\top w_2 &= c
\end{align}

In [68]:
K = return_K(D)

G_of_w = rotate_quadratic(G_of_z,(U/K)')
Q_of_w = rotate_quadratic(Q_of_z,(U/K)')

(
54x54 Array{Float64,2}:
 -3.1655e-22    6.42902e-22  -2.95008e-22  …   3.08591e-18   3.51695e-19
 -6.32953e-22  -1.18732e-21  -9.85102e-22      1.96146e-18   1.27949e-19
  8.97254e-23   4.61686e-21  -1.9837e-21       1.92169e-19  -3.06666e-19
 -1.46922e-21  -1.16393e-21   1.34214e-21     -9.44767e-19  -5.31573e-19
  2.81569e-21   1.09645e-21   1.3895e-22       1.90773e-18   1.08272e-18
  9.03974e-22  -1.98887e-21   1.61209e-21  …  -1.12167e-18   1.14232e-18
 -4.93817e-22   2.01697e-21  -4.6876e-22       1.65762e-18  -3.13352e-19
  5.6677e-22    1.52363e-21   2.13615e-21      4.23676e-19   1.25812e-18
  2.25058e-21   4.45724e-22  -1.21819e-21      1.97259e-18  -1.4752e-18 
 -8.67591e-22  -9.32033e-22  -2.60927e-21     -6.91245e-19  -3.91314e-19
 -2.2538e-22   -4.77284e-22  -2.554e-21    …   7.75556e-19  -2.19673e-19
  8.48507e-22   3.28066e-22  -3.42608e-24     -7.2416e-20    2.34875e-19
  4.5347e-22    1.17648e-21   6.82619e-22     -6.39781e-19  -3.71642e-19
  ⋮                      

In [86]:
Db,Vb = eig(G_of_w[1])

(Complex{Float64}[2864.67+0.0im,2864.67+0.0im,2864.67+0.0im,0.0525676+0.0im,0.0525676+0.0im,0.0525676+0.0im,0.504273+0.0im,0.504273+0.0im,0.504273+0.0im,0.83567+0.0im  …  0.999945+0.0im,0.999977+1.52428e-15im,0.999977-1.52428e-15im,0.999977+0.0im,0.999979+2.72344e-15im,0.999979-2.72344e-15im,0.999979+0.0im,0.99998+0.0im,0.99998+0.0im,0.99998+0.0im],
54x54 Array{Complex{Float64},2}:
    4.6132e-5+0.0im  -0.000396597+0.0im  …     0.184571+0.0im
 -0.000265281+0.0im  -0.000247767+0.0im      -0.0574623+0.0im
  0.000132332+0.0im  -0.000515375+0.0im      -0.0394737+0.0im
   3.75249e-6+0.0im  -0.000122041+0.0im      -0.0772261+0.0im
   7.26835e-7+0.0im   -3.12092e-5+0.0im       -0.156061+0.0im
    4.4186e-5+0.0im   0.000726628+0.0im  …   -0.0942749+0.0im
  0.000287455+0.0im   0.000281328+0.0im      -0.0343245+0.0im
 -0.000227179+0.0im    1.41191e-5+0.0im        0.230919+0.0im
  0.000321349+0.0im   -9.24697e-5+0.0im        0.173779+0.0im
  -0.00017924+0.0im   -7.46083e-5+0.0im       0.0785277+0

In [151]:
function return_B(Q̂obj,K,U)
    return K\(U'*Q̂obj*U)/K
end

return_B (generic function with 1 method)

In [158]:
using PyPlot

In [162]:
B = return_B(Q̂obj,K,U)

54x54 Array{Float64,2}:
  0.915449     -0.0233053   -0.0159132    …     0.998021      0.361789 
 -0.0233053     0.570312     0.050948           0.578915      0.964483 
 -0.0159132     0.050948     0.910002           1.03128       1.0733   
  0.00178841    0.0213245   -0.0447988          0.241112      0.320055 
 -0.0134943    -0.138397     0.0484468          0.0532128     0.106174 
  0.0116372     0.138438    -0.0124056    …    -1.86381      -0.830732 
 -0.0508072    -0.0471212    0.0114364         -0.856048     -0.502389 
 -0.00857514   -0.00219041   0.000919932        0.47634      -0.996263 
  0.000722838   0.0840308   -0.00730951        -0.0679        0.29139  
  0.0124101    -0.0515584    0.0482382          0.570232     -0.626403 
 -0.0159943    -0.00751549  -0.0473313    …    -0.777405      0.865928 
 -0.0433367    -0.0936823    0.0362432          0.124139     -0.293271 
  0.0132218     0.00593932  -0.0031133          0.458284      0.181731 
  ⋮                                     

Now find the Schur complement of 

In [174]:
find(D)

3-element Array{Int64,1}:
 52
 53
 54

In [178]:
function partition_B(B,D)
    D_zeros = setdiff(1:size(D,1),find(D))
    D_ones = find(D)
    B11 = B[D_zeros,D_zeros]
    B12 = B[D_zeros,D_ones]
    B21 = B[D_ones,D_zeros]
    B22 = B[D_ones,D_ones]
    return B11,B12,B21,B22
end

3x3 Array{Float64,2}:
 2864.66            3.95666e-12    -3.51906e-12
    3.79398e-12  2864.66           -5.14535e-12
   -3.29849e-12    -4.76563e-12  2864.66       

In [185]:
B_hat = B22 - B12'*(B11\B12)

3x3 Array{Float64,2}:
 2796.47            1.32522e-11    -7.65484e-12
    1.30861e-11  2796.47           -1.12583e-11
   -7.41611e-12    -1.08545e-11  2796.47       

In [186]:
D,U = eig(B_hat)

([2796.47,2796.47,2796.47],
3x3 Array{Float64,2}:
  0.476873  0.762512  -0.465289
  0.397031  0.290901   0.872576
 -0.784193  0.577886   0.14872 )

In [94]:
# Kernel rotation matrix:
Rkernel = kernel_rotation(A)

Rkernel = Rkernel[:,1:size(A,2) - rank(A)]'

# Rotate by Rkernel:
I_of_x = rotate(H_of_x,Rkernel)
S_of_x = rotate(R_of_x,Rkernel)

(
54x54 Array{Float64,2}:
  8.57715e-6  -7.83568e-6   8.32729e-6  …  -4.07499e-6  -9.85393e-6
 -7.83568e-6   1.91934e-5   5.16678e-6      3.09765e-6   1.42626e-5
  8.32729e-6   5.16678e-6   3.24455e-5     -2.57993e-6  -2.01826e-6
 -1.53017e-6   1.33535e-5   8.37457e-6     -4.28291e-7   6.46892e-6
  3.71994e-7   1.28016e-5   2.91055e-7     -3.50642e-6   2.76647e-6
 -2.11083e-7   1.47391e-5  -1.14544e-5  …  -5.69497e-6   1.74543e-6
 -2.1289e-6    1.01688e-5   2.87208e-5      4.74971e-6   1.00533e-5
  3.44011e-6   8.8652e-6    3.51056e-6     -4.63256e-6  -9.91135e-7
 -3.34661e-6   1.28924e-5   7.6256e-6       1.16143e-6   8.22289e-6
 -4.765e-6     1.32255e-5   1.22824e-5      3.21765e-6   1.07152e-5
 -1.75752e-6   9.48476e-6   2.5917e-5   …   4.06274e-6   8.96682e-6
 -2.55424e-6   1.02789e-5   1.16742e-5      1.88109e-6   7.44803e-6
 -7.16027e-6   1.42599e-5   9.35653e-6      4.53344e-6   1.30762e-5
  ⋮                                     ⋱                          
 -1.52987e-6  -1.7632e

In [ ]:
# R_of_x is rotation invariant.
J_of_z,Rkernel = tr_kernel_rotate(H_of_x,A)
K_of_w,Reigvec = tr_diag_rotate(J_of_z)

#H_of_x,J_of_z,K_of_w,R_of_x,x_star,Rkernel,Reigvec,A1,A2 = tr_trans_rotate(G_of_x,Q_of_x,A,b,Qtheta)

# Prepare to solve the secular equation:
#Qtheta = full(R_of_x[1])
# Qtheta = Reigvec*Rkernel*full(R_of_x[1])*Rkernel'*Reigvec'
D = K_of_w[1]
d = K_of_w[2]./2
d2 = 0#Reigvec*Rkernel*R_of_x[2]
c = -R_of_x[3]